In [1]:
import pandas as pd
import nltk
import numpy as np

# Get Data

In [2]:
review_reader = pd.read_json('yelp_academic_dataset_review.json', chunksize=1, lines=True)
i = 0
business_data = pd.DataFrame([])
for chunk in review_reader:
    #print(chunk)
    business_data = chunk
    i = i+1
    if i==1:
        break
#business_data = pd.concat(df)
#print(type(chunk))
business_data

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,iCQpiavjjPzJ5_3gPD5Ebg,0,2011-02-25,0,x7mDIiDB3jEiPGPHOmDzyw,2,The pizza was okay. Not the best I've had. I p...,0,msQe1u7Z_XuqjGoqhB0J5g


In [3]:
business_id = business_data['business_id']
business_id = business_id[0]
business_id

'iCQpiavjjPzJ5_3gPD5Ebg'

In [4]:
'''review_reader = pd.read_json('yelp_academic_dataset_review.json', chunksize=1, lines=True)
i = 0
j = 0
df = []
for chunk in review_reader:
    #for c in chunk:
    if chunk['business_id'][j] == business_id:
        #print(chunk['business_id'][j])
        df.append(chunk)
        i = i+1
        if i==200:
            break
    j = j+1
    #if j==10000:
    #    break'''

"review_reader = pd.read_json('yelp_academic_dataset_review.json', chunksize=1, lines=True)\ni = 0\nj = 0\ndf = []\nfor chunk in review_reader:\n    #for c in chunk:\n    if chunk['business_id'][j] == business_id:\n        #print(chunk['business_id'][j])\n        df.append(chunk)\n        i = i+1\n        if i==200:\n            break\n    j = j+1\n    #if j==10000:\n    #    break"

In [5]:
#review_data = pd.concat(df)
review_data = pd.read_csv('review_data.csv')
review_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 395 entries, 0 to 394
Data columns (total 10 columns):
Unnamed: 0     395 non-null int64
business_id    395 non-null object
cool           395 non-null int64
date           395 non-null object
funny          395 non-null int64
review_id      395 non-null object
stars          395 non-null int64
text           395 non-null object
useful         395 non-null int64
user_id        395 non-null object
dtypes: int64(5), object(5)
memory usage: 30.9+ KB


## Write data to csv file

In [6]:
#review_data.to_csv('review_data.csv')
#review_text.to_json('review_text.json')

## Cleaning data

In [7]:
reviews = review_data.loc[:,['review_id','text']]
reviews.head()

,review_id,text
0,x7mDIiDB3jEiPGPHOmDzyw,The pizza was okay. Not the best I've had. I p...
1,VluIpojME0yKOcRKI5L0Iw,"came here on Monday, no line. its good, but no..."
2,nsB9JAeghk0zOaSulSm9Yw,This place is truly a secret! Its so hidden t...
3,pcn01EMERpCpHEcoaohdEg,This SECRET PIZZA was a secret for about 5 st...
4,0mFtAPTmInbXHqDjX9eiOg,this is the BEST place to grab a quick slice o...


In [8]:
#import re
txt = []
for index,row in reviews.iterrows():
    text = row['text']
    text = text.split('. ')
    txt.extend(text)
txt
review_text = pd.DataFrame(txt, columns=['text'])
review_text.head()

,text
0,The pizza was okay
1,Not the best I've had
2,I prefer Biaggio's on Flamingo / Fort Apache
3,The chef there can make a MUCH better NY style...
4,The pizzeria @ Cosmo was over priced for the q...


# Tokenize

In [9]:
#nltk.download('punkt')
token_list = []
for index,row in review_text.iterrows():
    text = row['text'].lower()
    tokens = nltk.word_tokenize(text)
    token_list.append([tokens])
    #print(token_list)
token_col = pd.DataFrame(token_list, columns=['tokens'])
#print(token_list)
#token_col.head()
review_text = pd.concat([review_text, token_col], axis=1)
review_text.head()

,text,tokens
0,The pizza was okay,"[the, pizza, was, okay]"
1,Not the best I've had,"[not, the, best, i, 've, had]"
2,I prefer Biaggio's on Flamingo / Fort Apache,"[i, prefer, biaggio, 's, on, flamingo, /, fort..."
3,The chef there can make a MUCH better NY style...,"[the, chef, there, can, make, a, much, better,..."
4,The pizzeria @ Cosmo was over priced for the q...,"[the, pizzeria, @, cosmo, was, over, priced, f..."


# 3.1 POS Tagging

## Stopword removal and conversion to lowercase
### (not needed)

In [10]:
'''#nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english')) 
ftext = []
for index,row in review_text.iterrows():
    tokens = row['tokens']
    filtered_text = [w.lower() for w in tokens]#if not w in stop_words] 
    ftext.append([filtered_text])
filtered_col = pd.DataFrame(ftext, columns=['filtered_tokens'])
review_text = pd.concat([review_text, filtered_col], axis=1)
review_text.head()'''

"#nltk.download('stopwords')\nfrom nltk.corpus import stopwords\nstop_words = set(stopwords.words('english')) \nftext = []\nfor index,row in review_text.iterrows():\n    tokens = row['tokens']\n    filtered_text = [w.lower() for w in tokens]#if not w in stop_words] \n    ftext.append([filtered_text])\nfiltered_col = pd.DataFrame(ftext, columns=['filtered_tokens'])\nreview_text = pd.concat([review_text, filtered_col], axis=1)\nreview_text.head()"

## Lemmatizing

In [11]:
'''from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
ttext = []
for index,row in review_text.iterrows():
    filtered_text = row['tokens']
    lemmatized_tokens = []
    for ftr in filtered_text:
        lf = lemmatizer.lemmatize(ftr)
        lemmatized_tokens.append(lf)
    ttext.append([lemmatized_tokens])
tagged_col = pd.DataFrame(ttext, columns=['lemmatized_tokens'])
review_text = pd.concat([review_text, tagged_col], axis=1)
review_text.head()'''

"from nltk.stem import WordNetLemmatizer\nlemmatizer = WordNetLemmatizer()\nttext = []\nfor index,row in review_text.iterrows():\n    filtered_text = row['tokens']\n    lemmatized_tokens = []\n    for ftr in filtered_text:\n        lf = lemmatizer.lemmatize(ftr)\n        lemmatized_tokens.append(lf)\n    ttext.append([lemmatized_tokens])\ntagged_col = pd.DataFrame(ttext, columns=['lemmatized_tokens'])\nreview_text = pd.concat([review_text, tagged_col], axis=1)\nreview_text.head()"

## Tagging using upenn tagset

In [12]:
#nltk.download('averaged_perceptron_tagger')
ttext = []
for index,row in review_text.iterrows():
    filtered_text = row['tokens']
    tagged_text = nltk.pos_tag(filtered_text) 
    ttext.append([tagged_text])
tagged_col = pd.DataFrame(ttext, columns=['tagged_tokens'])
review_text = pd.concat([review_text, tagged_col], axis=1)
review_text.head()

,text,tokens,tagged_tokens
0,The pizza was okay,"[the, pizza, was, okay]","[(the, DT), (pizza, NN), (was, VBD), (okay, JJ)]"
1,Not the best I've had,"[not, the, best, i, 've, had]","[(not, RB), (the, DT), (best, JJS), (i, NN), (..."
2,I prefer Biaggio's on Flamingo / Fort Apache,"[i, prefer, biaggio, 's, on, flamingo, /, fort...","[(i, NN), (prefer, VBP), (biaggio, NN), ('s, P..."
3,The chef there can make a MUCH better NY style...,"[the, chef, there, can, make, a, much, better,...","[(the, DT), (chef, NN), (there, EX), (can, MD)..."
4,The pizzeria @ Cosmo was over priced for the q...,"[the, pizzeria, @, cosmo, was, over, priced, f...","[(the, DT), (pizzeria, NN), (@, NNP), (cosmo, ..."


In [13]:
#nltk.download('tagsets')
#nltk.help.upenn_tagset('NN')

## Noun / noun phrase filtering

In [14]:
ntext = []
for index,row in review_text.iterrows():
    tagged_text = row['tagged_tokens']
    nouns = []
    for tt in tagged_text:
        tag = tt[1]
        if tag in ['NN','NNS','NNP','NNPS']:
            nouns.append(tt[0])
    ntext.append([nouns])
noun_col = pd.DataFrame(ntext, columns=['nouns'])
noun_col
review_text = pd.concat([review_text, noun_col], axis=1)
review_text.head()

,text,tokens,tagged_tokens,nouns
0,The pizza was okay,"[the, pizza, was, okay]","[(the, DT), (pizza, NN), (was, VBD), (okay, JJ)]",[pizza]
1,Not the best I've had,"[not, the, best, i, 've, had]","[(not, RB), (the, DT), (best, JJS), (i, NN), (...",[i]
2,I prefer Biaggio's on Flamingo / Fort Apache,"[i, prefer, biaggio, 's, on, flamingo, /, fort...","[(i, NN), (prefer, VBP), (biaggio, NN), ('s, P...","[i, biaggio, flamingo, /, fort, apache]"
3,The chef there can make a MUCH better NY style...,"[the, chef, there, can, make, a, much, better,...","[(the, DT), (chef, NN), (there, EX), (can, MD)...","[chef, style, pizza]"
4,The pizzeria @ Cosmo was over priced for the q...,"[the, pizzeria, @, cosmo, was, over, priced, f...","[(the, DT), (pizzeria, NN), (@, NNP), (cosmo, ...","[pizzeria, @, cosmo, quality, lack, personalit..."


## Stemming 
### (not including in procedure)

In [15]:
'''from nltk.stem import PorterStemmer
ps = PorterStemmer()
for w in nouns:
    print(w + '\t:\t' + ps.stem(w))'''

"from nltk.stem import PorterStemmer\nps = PorterStemmer()\nfor w in nouns:\n    print(w + '\t:\t' + ps.stem(w))"

## Fuzzy matching (example)
### (not including in procedure)

In [16]:
'''from fuzzywuzzy import fuzz
from fuzzywuzzy import process
query = 'Barack Obama'
choices = ['Barack H Obama', 'Barack H. Obama', 'B. Obama']
# Get a list of matches ordered by score, default limit to 5
process.extract(query, choices)
# [('Barack H Obama', 95), ('Barack H. Obama', 95), ('B. Obama', 85)]
 
# If we want only the top one
process.extractOne(query, choices)
# ('Barack H Obama', 95)'''

"from fuzzywuzzy import fuzz\nfrom fuzzywuzzy import process\nquery = 'Barack Obama'\nchoices = ['Barack H Obama', 'Barack H. Obama', 'B. Obama']\n# Get a list of matches ordered by score, default limit to 5\nprocess.extract(query, choices)\n# [('Barack H Obama', 95), ('Barack H. Obama', 95), ('B. Obama', 85)]\n \n# If we want only the top one\nprocess.extractOne(query, choices)\n# ('Barack H Obama', 95)"

## Writing processed data to CSV

In [17]:
review_text.to_csv('review_text.csv')

## Running additional noun filtering
### (not needed)

In [18]:
'''ttext = []
for index,row in review_text.iterrows():
    filtered_text = row['nouns']
    tagged_text = nltk.pos_tag(filtered_text) 
    ttext.append([tagged_text])
tagged_col = pd.DataFrame(ttext, columns=['tagged_noun_tokens'])
review_text = pd.concat([review_text, tagged_col], axis=1)
review_text.head()'''

"ttext = []\nfor index,row in review_text.iterrows():\n    filtered_text = row['nouns']\n    tagged_text = nltk.pos_tag(filtered_text) \n    ttext.append([tagged_text])\ntagged_col = pd.DataFrame(ttext, columns=['tagged_noun_tokens'])\nreview_text = pd.concat([review_text, tagged_col], axis=1)\nreview_text.head()"

In [19]:
'''ntext = []
for index,row in review_text.iterrows():
    tagged_text = row['tagged_noun_tokens']
    nouns = []
    for tt in tagged_text:
        tag = tt[1]
        if tag in ['NN','NNS','NNP','NNPS']:
            nouns.append(tt[0])
    ntext.append([nouns])
noun_col = pd.DataFrame(ntext, columns=['filtered_nouns'])
noun_col
review_text = pd.concat([review_text, noun_col], axis=1)
review_text.head()'''

"ntext = []\nfor index,row in review_text.iterrows():\n    tagged_text = row['tagged_noun_tokens']\n    nouns = []\n    for tt in tagged_text:\n        tag = tt[1]\n        if tag in ['NN','NNS','NNP','NNPS']:\n            nouns.append(tt[0])\n    ntext.append([nouns])\nnoun_col = pd.DataFrame(ntext, columns=['filtered_nouns'])\nnoun_col\nreview_text = pd.concat([review_text, noun_col], axis=1)\nreview_text.head()"

# 3.2 Frequent features identification

In [20]:
filtered_nouns = review_text['nouns']
filtered_nouns = filtered_nouns.tolist()

In [21]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
lemm_nouns = []
for fn in filtered_nouns:
    lf = []
    for f in fn:
        lf.append(lemmatizer.lemmatize(f))
    lemm_nouns.append(lf)
lemm_nouns[0:5]

[['pizza'],
 ['i'],
 ['i', 'biaggio', 'flamingo', '/', 'fort', 'apache'],
 ['chef', 'style', 'pizza'],
 ['pizzeria', '@', 'cosmo', 'quality', 'lack', 'personality', 'food']]

In [22]:
features = {}
#print(features.keys())
for nouns in lemm_nouns:
    keys = features.keys()
    visited = []
    for n in nouns:
        if n not in visited:
            if n in keys:
                features[n] += 1
            else:
                features.update({n: 1})

#features

In [23]:
frequent_ftr = []
print(review_text['text'].count())
threshold = int(review_text['text'].count() * 0.015)
for k,v in features.items():
    if v >= threshold:
        frequent_ftr.append(k)
frequent_ftr

1867


['spot',
 'strip',
 'floor',
 'vega',
 'minute',
 'topping',
 'price',
 'crust',
 'sausage',
 'thing',
 'ricotta',
 'i',
 'night',
 'slice',
 'staff',
 'area',
 'pizza',
 'restaurant',
 'cosmo',
 'place',
 'order',
 'wait',
 'york',
 'service',
 'cosmopolitan',
 'hallway',
 'line',
 'sign',
 'style',
 'way',
 'friend',
 'pepperoni',
 'pie',
 'food',
 'time',
 'cheese',
 'people']

# 3.3 Opinion words extraction

In [24]:
'''opinion_words = {}
for index,row in review_text.iterrows():
    op_words = []
    for ftr in frequent_ftr:
        if ftr in row['tokens']:
            for tt in row['tagged_tokens']:
                tag = tt[1]
                word = tt[0]
                if (tag in ['JJ','JJR','JJS']) and (word not in op_words):
                    op_words.append(tt[0])
    opinion_words.update({ftr:op_words})

opinion_words'''

"opinion_words = {}\nfor index,row in review_text.iterrows():\n    op_words = []\n    for ftr in frequent_ftr:\n        if ftr in row['tokens']:\n            for tt in row['tagged_tokens']:\n                tag = tt[1]\n                word = tt[0]\n                if (tag in ['JJ','JJR','JJS']) and (word not in op_words):\n                    op_words.append(tt[0])\n    opinion_words.update({ftr:op_words})\n\nopinion_words"

# Sentiment analysis

In [25]:
#nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [26]:
import random
from nltk.corpus import movie_reviews

documents = [(list(movie_reviews.words(fileid)), category)
             for category in movie_reviews.categories()
             for fileid in movie_reviews.fileids(category)]

random.shuffle(documents)

#print(documents[1])

all_words = []
for w in movie_reviews.words():
    all_words.append(w.lower())

all_words = nltk.FreqDist(all_words)

In [27]:
word_features = list(all_words.keys())[:3000]

def find_features(document):
    words = set(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)

    return features

#print((find_features(movie_reviews.words('neg/cv000_29416.txt'))))
featuresets = [(find_features(rev), category) for (rev, category) in documents]

In [29]:
# set that we'll train our classifier with
training_set = featuresets

# set that we'll test against.
#testing_set = featuresets[1900:]
dat1 = []
for index,row in review_text.iterrows():
    words = row['tokens']
    dic = ({w:True for w in words}, 'pos')
    dat1.append(dic)

testing_set = dat1

classifier = nltk.NaiveBayesClassifier.train(training_set)
print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier, testing_set))*100)
classifier.show_most_informative_features(15)

Classifier accuracy percent: 71.77289769683985
Most Informative Features
                    slip = True              pos : neg    =     11.7 : 1.0
                  feeble = True              neg : pos    =      8.3 : 1.0
                 symbols = True              pos : neg    =      7.7 : 1.0
                 idiotic = True              neg : pos    =      7.4 : 1.0
                supports = True              pos : neg    =      7.0 : 1.0
                    taxi = True              pos : neg    =      6.6 : 1.0
                  skimpy = True              neg : pos    =      6.3 : 1.0
                  elmore = True              pos : neg    =      6.3 : 1.0
                   masks = True              neg : pos    =      5.7 : 1.0
                  purple = True              pos : neg    =      5.7 : 1.0
                 kidnaps = True              neg : pos    =      5.7 : 1.0
              dedication = True              pos : neg    =      5.7 : 1.0
                 warrant = 

In [30]:
import pickle
save_classifier = open("naivebayes.pickle","wb")
pickle.dump(classifier, save_classifier)
save_classifier.close()

In [31]:
classifier_f = open("naivebayes.pickle", "rb")
classifier = pickle.load(classifier_f)
print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier, testing_set))*100)
classifier.show_most_informative_features(15)
classifier_f.close()

Classifier accuracy percent: 71.77289769683985
Most Informative Features
                    slip = True              pos : neg    =     11.7 : 1.0
                  feeble = True              neg : pos    =      8.3 : 1.0
                 symbols = True              pos : neg    =      7.7 : 1.0
                 idiotic = True              neg : pos    =      7.4 : 1.0
                supports = True              pos : neg    =      7.0 : 1.0
                    taxi = True              pos : neg    =      6.6 : 1.0
                  skimpy = True              neg : pos    =      6.3 : 1.0
                  elmore = True              pos : neg    =      6.3 : 1.0
                   masks = True              neg : pos    =      5.7 : 1.0
                  purple = True              pos : neg    =      5.7 : 1.0
                 kidnaps = True              neg : pos    =      5.7 : 1.0
              dedication = True              pos : neg    =      5.7 : 1.0
                 warrant = 

In [32]:
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(training_set)
print("MultinomialNB accuracy percent:",nltk.classify.accuracy(MNB_classifier, testing_set))

BNB_classifier = SklearnClassifier(BernoulliNB())
BNB_classifier.train(training_set)
print("BernoulliNB accuracy percent:",nltk.classify.accuracy(BNB_classifier, testing_set))

MultinomialNB accuracy percent: 0.20085698982324585
BernoulliNB accuracy percent: 0.05088377075522228


In [33]:
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.svm import SVC, LinearSVC, NuSVC
print("Original Naive Bayes Algo accuracy percent:", (nltk.classify.accuracy(classifier, testing_set))*100)
classifier.show_most_informative_features(15)

MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(training_set)
print("MNB_classifier accuracy percent:", (nltk.classify.accuracy(MNB_classifier, testing_set))*100)

BernoulliNB_classifier = SklearnClassifier(BernoulliNB())
BernoulliNB_classifier.train(training_set)
print("BernoulliNB_classifier accuracy percent:", (nltk.classify.accuracy(BernoulliNB_classifier, testing_set))*100)

LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier.train(training_set)
print("LogisticRegression_classifier accuracy percent:", (nltk.classify.accuracy(LogisticRegression_classifier, testing_set))*100)

SGDClassifier_classifier = SklearnClassifier(SGDClassifier())
SGDClassifier_classifier.train(training_set)
print("SGDClassifier_classifier accuracy percent:", (nltk.classify.accuracy(SGDClassifier_classifier, testing_set))*100)

SVC_classifier = SklearnClassifier(SVC())
SVC_classifier.train(training_set)
print("SVC_classifier accuracy percent:", (nltk.classify.accuracy(SVC_classifier, testing_set))*100)

LinearSVC_classifier = SklearnClassifier(LinearSVC())
LinearSVC_classifier.train(training_set)
print("LinearSVC_classifier accuracy percent:", (nltk.classify.accuracy(LinearSVC_classifier, testing_set))*100)

NuSVC_classifier = SklearnClassifier(NuSVC())
NuSVC_classifier.train(training_set)
print("NuSVC_classifier accuracy percent:", (nltk.classify.accuracy(NuSVC_classifier, testing_set))*100)

Original Naive Bayes Algo accuracy percent: 71.77289769683985
Most Informative Features
                    slip = True              pos : neg    =     11.7 : 1.0
                  feeble = True              neg : pos    =      8.3 : 1.0
                 symbols = True              pos : neg    =      7.7 : 1.0
                 idiotic = True              neg : pos    =      7.4 : 1.0
                supports = True              pos : neg    =      7.0 : 1.0
                    taxi = True              pos : neg    =      6.6 : 1.0
                  skimpy = True              neg : pos    =      6.3 : 1.0
                  elmore = True              pos : neg    =      6.3 : 1.0
                   masks = True              neg : pos    =      5.7 : 1.0
                  purple = True              pos : neg    =      5.7 : 1.0
                 kidnaps = True              neg : pos    =      5.7 : 1.0
              dedication = True              pos : neg    =      5.7 : 1.0
            

/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


SGDClassifier_classifier accuracy percent: 69.8446705945367
SVC_classifier accuracy percent: 94.32244242099625
LinearSVC_classifier accuracy percent: 80.02142474558114
NuSVC_classifier accuracy percent: 20.032137118371722


In [34]:
from nltk.classify import ClassifierI
from statistics import mode

class VoteClassifier(ClassifierI):
    def __init__(self, *classifiers):
        self._classifiers = classifiers

    def classify(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        return mode(votes)
    
    def confidence(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)

        choice_votes = votes.count(mode(votes))
        conf = choice_votes / len(votes)
        return conf

In [35]:
classifier_f = open("naivebayes.pickle","rb")
classifier = pickle.load(classifier_f)
classifier_f.close()




print("Original Naive Bayes Algo accuracy percent:", (nltk.classify.accuracy(classifier, testing_set))*100)
classifier.show_most_informative_features(15)

MNB_classifier = SklearnClassifier(MultinomialNB())
MNB_classifier.train(training_set)
print("MNB_classifier accuracy percent:", (nltk.classify.accuracy(MNB_classifier, testing_set))*100)

BernoulliNB_classifier = SklearnClassifier(BernoulliNB())
BernoulliNB_classifier.train(training_set)
print("BernoulliNB_classifier accuracy percent:", (nltk.classify.accuracy(BernoulliNB_classifier, testing_set))*100)

LogisticRegression_classifier = SklearnClassifier(LogisticRegression())
LogisticRegression_classifier.train(training_set)
print("LogisticRegression_classifier accuracy percent:", (nltk.classify.accuracy(LogisticRegression_classifier, testing_set))*100)

SGDClassifier_classifier = SklearnClassifier(SGDClassifier())
SGDClassifier_classifier.train(training_set)
print("SGDClassifier_classifier accuracy percent:", (nltk.classify.accuracy(SGDClassifier_classifier, testing_set))*100)

##SVC_classifier = SklearnClassifier(SVC())
##SVC_classifier.train(training_set)
##print("SVC_classifier accuracy percent:", (nltk.classify.accuracy(SVC_classifier, testing_set))*100)

LinearSVC_classifier = SklearnClassifier(LinearSVC())
LinearSVC_classifier.train(training_set)
print("LinearSVC_classifier accuracy percent:", (nltk.classify.accuracy(LinearSVC_classifier, testing_set))*100)

NuSVC_classifier = SklearnClassifier(NuSVC())
NuSVC_classifier.train(training_set)
print("NuSVC_classifier accuracy percent:", (nltk.classify.accuracy(NuSVC_classifier, testing_set))*100)


voted_classifier = VoteClassifier(classifier,
                                  NuSVC_classifier,
                                  LinearSVC_classifier,
                                  SGDClassifier_classifier,
                                  MNB_classifier,
                                  BernoulliNB_classifier,
                                  LogisticRegression_classifier)

print("voted_classifier accuracy percent:", (nltk.classify.accuracy(voted_classifier, testing_set))*100)

print("Classification:", voted_classifier.classify(testing_set[0][0]), "Confidence %:",voted_classifier.confidence(testing_set[0][0])*100)
print("Classification:", voted_classifier.classify(testing_set[1][0]), "Confidence %:",voted_classifier.confidence(testing_set[1][0])*100)
print("Classification:", voted_classifier.classify(testing_set[2][0]), "Confidence %:",voted_classifier.confidence(testing_set[2][0])*100)
print("Classification:", voted_classifier.classify(testing_set[3][0]), "Confidence %:",voted_classifier.confidence(testing_set[3][0])*100)
print("Classification:", voted_classifier.classify(testing_set[4][0]), "Confidence %:",voted_classifier.confidence(testing_set[4][0])*100)
print("Classification:", voted_classifier.classify(testing_set[5][0]), "Confidence %:",voted_classifier.confidence(testing_set[5][0])*100)

Original Naive Bayes Algo accuracy percent: 71.77289769683985
Most Informative Features
                    slip = True              pos : neg    =     11.7 : 1.0
                  feeble = True              neg : pos    =      8.3 : 1.0
                 symbols = True              pos : neg    =      7.7 : 1.0
                 idiotic = True              neg : pos    =      7.4 : 1.0
                supports = True              pos : neg    =      7.0 : 1.0
                    taxi = True              pos : neg    =      6.6 : 1.0
                  skimpy = True              neg : pos    =      6.3 : 1.0
                  elmore = True              pos : neg    =      6.3 : 1.0
                   masks = True              neg : pos    =      5.7 : 1.0
                  purple = True              pos : neg    =      5.7 : 1.0
                 kidnaps = True              neg : pos    =      5.7 : 1.0
              dedication = True              pos : neg    =      5.7 : 1.0
            

/usr/local/lib/python3.5/dist-packages/sklearn/linear_model/stochastic_gradient.py:128: FutureWarning: max_iter and tol parameters have been added in <class 'sklearn.linear_model.stochastic_gradient.SGDClassifier'> in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  "and default tol will be 1e-3." % type(self), FutureWarning)


SGDClassifier_classifier accuracy percent: 28.602035350830207
LinearSVC_classifier accuracy percent: 80.02142474558114
NuSVC_classifier accuracy percent: 20.032137118371722
voted_classifier accuracy percent: 22.603106588109267
Classification: neg Confidence %: 57.14285714285714
Classification: neg Confidence %: 57.14285714285714
Classification: neg Confidence %: 100.0
Classification: neg Confidence %: 71.42857142857143
Classification: neg Confidence %: 57.14285714285714
Classification: neg Confidence %: 100.0


In [ ]:
'''from nltk import word_tokenize
short_pos = open("positive.txt","r").read()
short_neg = open("negative.txt","r").read()

documents = []

for r in short_pos.split('\n'):
    documents.append( (r, "pos") )

for r in short_neg.split('\n'):
    documents.append( (r, "neg") )


all_words = []

short_pos_words = word_tokenize(short_pos)
short_neg_words = word_tokenize(short_neg)

for w in short_pos_words:
    all_words.append(w.lower())

for w in short_neg_words:
    all_words.append(w.lower())

all_words = nltk.FreqDist(all_words)'''